# Book Recommendation System

#### import libraries

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import codecs

## Load the data-sets

In [18]:
#### BX_Books.csv
books = pd.read_csv('BX_Books.csv',sep=';',encoding='latin-1')

In [19]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [16]:
#### BX-Book-Ratings.csv
book_ratings = pd.read_csv('BX-Book-Ratings.csv',sep=';',encoding='latin-1')

In [20]:
book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [17]:
#### BX-Users.csv
users = pd.read_csv('BX-Users.csv',sep=';',encoding='latin-1')

In [21]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


## Merging the data-sets

In [23]:
new_user = pd.merge(users,book_ratings,on='User-ID')

In [24]:
merged_data_set = pd.merge(books,new_user,on='ISBN')

In [25]:
merged_data_set.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Location,Age,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,"stockton, california, usa",18.0,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,"timmins, ontario, canada",NaN,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,"ottawa, ontario, canada",49.0,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,"n/a, n/a, n/a",NaN,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,"sudbury, ontario, canada",NaN,0


## EDA

In [29]:
merged_data_set.shape

(1031175, 12)

In [26]:
merged_data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031175 entries, 0 to 1031174
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ISBN                 1031175 non-null  object 
 1   Book-Title           1031175 non-null  object 
 2   Book-Author          1031173 non-null  object 
 3   Year-Of-Publication  1031175 non-null  int64  
 4   Publisher            1031173 non-null  object 
 5   Image-URL-S          1031175 non-null  object 
 6   Image-URL-M          1031175 non-null  object 
 7   Image-URL-L          1031175 non-null  object 
 8   User-ID              1031175 non-null  int64  
 9   Location             1031175 non-null  object 
 10  Age                  753330 non-null   float64
 11  Book-Rating          1031175 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 94.4+ MB


In [27]:
merged_data_set.describe()

,Year-Of-Publication,User-ID,Age,Book-Rating
count,1.031175e+06,1.031175e+06,753330.000000,1.031175e+06
mean,1.968194e+03,1.405944e+05,37.397719,2.839022e+00
std,2.311048e+02,8.052444e+04,14.098189,3.854149e+00
min,0.000000e+00,2.000000e+00,0.000000,0.000000e+00
25%,1.992000e+03,7.041500e+04,28.000000,0.000000e+00
50%,1.997000e+03,1.412100e+05,35.000000,0.000000e+00
75%,2.001000e+03,2.114260e+05,45.000000,7.000000e+00
max,2.050000e+03,2.788540e+05,244.000000,1.000000e+01


In [30]:
merged_data_set.isna().sum()

ISBN                        0
Book-Title                  0
Book-Author                 2
Year-Of-Publication         0
Publisher                   2
Image-URL-S                 0
Image-URL-M                 0
Image-URL-L                 0
User-ID                     0
Location                    0
Age                    277845
Book-Rating                 0
dtype: int64